<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/CreateAutoSentence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##RNNを使った文章の自動生成
https://www.pytry3g.com/entry/2018/03/16/203414

In [1]:
!pip install beautifulsoup4
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 1.5MB/s 


In [0]:
import MeCab
import re
import requests
from bs4 import BeautifulSoup

In [21]:
### データの用意 ###
# Wikipediaの記事をダウンロード
url = "http://ja.wikipedia.org/wiki/"
# 記事のリスト
wiki_name_list = ["織田信長"]
# データを入れる
corpus = []
tagger = MeCab.Tagger('-Owakati')
for wiki_name in wiki_name_list:
    # 指定したページの情報を取得
    response = requests.get(url+wiki_name)
    # HTMLフォーマットに変換
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    # pタグで囲まれた部分を抽出
    for p_tag in soup.find_all('p'):
        sentences = p_tag.text.strip()
        for sentence in sentences.split("。"):
            if len(sentence.strip()) == 0:
                continue
            # 前処理
            sentence = re.sub(r"\[[0-9]+\]", "", sentence)
            sentence = re.sub(r"\[注釈 [0-9]+\]", "", sentence)
            tokens = tagger.parse(sentence).strip().split()
            if len(tokens) > 46:
                continue
            corpus.append(sentence+"。")
print(corpus[0])

織田 信長（おだ のぶなが、1534年-1582年）は、戦国時代から安土桃山時代にかけての武将、戦国大名、天下人。


In [0]:
import pandas as pd
df = pd.DataFrame(data=corpus)
df.to_csv("text.txt",index=False,header=None)

##マルコフ連鎖で文章作成
https://chishiki-motomeru.com/post/20190329/

In [17]:
!pip install janome

     |████████████████████████████████| 25.1MB 673kB/s 


In [26]:
from janome.tokenizer import Tokenizer
import re
import random

markov = {}
sentence = ''

#janomeを使用して文章をパースさせる。
def parse(text):
    
    t = Tokenizer()
    tokens = t.tokenize(text)
    result = []
    for token in tokens:
        result.append(token.surface)
    return(result)

#取得したファイルネームから、改行を除去した上でパースする関数を呼び出す。
#ワードリストに代入して、それを返却させる。
def get_morpheme(filename):

    with open(filename,"r",encoding = "utf_8") as f:
        text = f.read()
    text = re.sub("\n","",text)
    wordlist = parse(text)

    return wordlist

#マルコフ辞書を生成させるmarkovは辞書型
def create_markov(wordlist):

    p1 = ""
    p2 = ""
    p3 = ""
    
    #連続した単語に続く、次の単語をサフィックスとする。単語の並びが同じであればそこに二個目のサフィックスが追加される
    for word in wordlist:
        if p1 and p2 and p3:
            if (p1,p2,p3) not in markov:
                markov[(p1,p2,p3)] = []
            markov[(p1,p2,p3)].append(word)
        p1,p2,p3 = p2,p3,word

    #プレフィックスの3つに続くサフィックスを表示する。
    print("マルコフ辞書の中身:{}".format(markov))
    print()

#マルコフ辞書からsentenceに文字列を格納させる。
def generate(wordlist):

    global sentence

    #3つのプレフィックスをランダムに取得して、それをプレフィックスに代入させる。
    p1,p2,p3 = random.choice(list(markov.keys()))
    print("マルコフ連鎖のキー[{},{},{}]".format(p1,p2,p3))
    print()

    count = 0

    while count < len(wordlist):
        if ((p1,p2,p3) in markov ) == True:
            tmp = random.choice(markov[(p1,p2,p3)])
            sentence += tmp
        p1,p2,p3 = p2,p3,tmp
        count += 1

    sentence = re.sub("^.+?","",sentence)

    if re.search(".+。",sentence):
        sentence = re.search(".+。",sentence).group()

    sentence = re.sub("」","",sentence)
    sentence = re.sub("「","",sentence)
    sentence = re.sub("　","",sentence)

    print(sentence)

#重複した文章を取り除く。sprit("。")にすることで一文単位でリスト化できる。最終的にリストを文字列型に戻す。
#一文単位でリスト化しているので、ですますが連続した場合はここで対処できる。
def overlap():

    global  sentence

    sentence = sentence.split("。")

    print(sentence)

    #空白リストを除去する
    if "" in sentence:
        sentence.remove("")

    #この工程で一文の末尾に。を付ける。
    new = []
    for str in sentence:
        str = str + "。"
        if str == "。":
            break
        new.append(str)

    #set関数を使用して、文章の重複を取り除いている
    new = set(new)

    #set関数を使用しているので、元のリストと文章の順序が異なっている。
    #文章の順序についてはこの時点で修正したほうが良さそう
    print(new)
    sentence="".join(new)

if __name__ == "__main__":
    
    word_list = get_morpheme("text.txt")

    print("ワードリスト:{}".format(word_list))
    print()

    create_markov(word_list)

    while(not sentence):
        generate(word_list)
        overlap()

    print("文章",sentence)

ワードリスト:['0', '織田', ' ', '信長', '（', 'おだ', ' ', 'のぶ', 'な', 'が', '、', '1534', '年', '-', '1582', '年', '）', 'は', '、', '戦国', '時代', 'から', '安土', '桃山', '時代', 'にかけて', 'の', '武将', '、', '戦国', '大名', '、', '天下', '人', '。', '織田', '信長', 'は', '、', '織田', '弾正', '忠', '家', 'の', '当主', '・', '織田', '信秀', 'の', '子', 'に', '生まれ', '、', '尾張', '（', '愛知', '県', '西部', '）', 'の', '一', '地方', '領主', 'として', 'その', '生涯', 'を', '歩み', '始め', 'た', '。', '永', '禄', '3', '年', '（', '1560', '年', '）', '、', '信長', 'は', '桶', '狭間', 'の', '戦い', 'において', '駿河', 'の', '戦国', '大名', '・', '今川', '義元', 'を', '撃破', 'し', 'た', '。', 'そして', '、', '三河', 'の', '領主', '・', '徳川', '家康', '（', '松平', '元康', '）', 'と', '同盟', 'を', '結ぶ', '。', '永', '禄', '8', '年', '（', '1565', '年', '）', '、', '犬山', '城', 'の', '織田', '信清', 'を', '破る', 'こと', 'で', '尾張', 'の', '統一', 'を', '達成', 'し', 'た', '。', 'そして', '、', '改めて', '幕府', '再興', 'を', '志す', '意', 'を', '込め', 'て', '、', '「', '天下', '布', '武', '」', 'の', '印', 'を', '使用', 'し', 'た', '。', '翌年', '10', '月', '、', '足利', '義昭', 'とともに', '信長', 'は', '上洛', 'し', '、', '三好'